In [ ]:
import torch
import torch.sparse as sparse
from scipy.sparse import csr_matrix
import scipy.sparse
import scipy.sparse.linalg
import math
import numpy as np
import scipy 
import importlib
import time
import torch.sparse as sparse
from scipy.sparse import csr_matrix
import scipy.sparse
import scipy.sparse.linalg
import math
import numpy as np
import scipy 
import scipy.io
import matplotlib.pyplot as plt
from functools import partial
import time
import sys
import os
import gc
import importlib
import RFrtModel_AdvDiff_SolverDA as AdvDiff_DA

In [ ]:
importlib.reload(AdvDiff_DA)

In [ ]:
def cond_alpha(t):
    # conditional information
    # alpha_t(0) = 1
    # alpha_t(1) = esp_alpha \approx 0
    return 1 - (1-eps_alpha)*t


def cond_sigma_sq(t):
    # conditional sigma^2
    # sigma2_t(0) = 0
    # sigma2_t(1) = 1
    # sigma(t) = t
    return t

def cond_sigma_sq(t):
    # conditional sigma^2
    # sigma2_t(0) = 0
    # sigma2_t(1) = 1
    # sigma(t) = t
    return t

def f(t):
    # f=d_(log_alpha)/dt
    alpha_t = cond_alpha(t)
    f_t = -(1-eps_alpha) / alpha_t
    return f_t

def g_sq(t):
    # g = d(sigma_t^2)/dt -2f sigma_t^2
    d_sigma_sq_dt = 1
    g2 = d_sigma_sq_dt - 2*f(t)*cond_sigma_sq(t)
    return g2

def g(t):
    return np.sqrt(g_sq(t))

def reverse_SDE(n_dim, select_set, idA_sub, x0, time_steps, C, score_likelihood=None, drift_fun=f, \
                diffuse_fun=g, alpha_fun=cond_alpha, sigma2_fun=cond_sigma_sq, save_path=False):
    # x_T: sample from standard Gaussian
    # x_0: target distribution to sample from

    # reverse SDE sampling process
    # N1 = x_T.shape[0]
    # N2 = x0.shape[0]
    # d = x_T.shape[1]

    # Generate the time mesh
    dt = 1.0/time_steps

    # Initialization
    xt = torch.randn(x0.shape[0], x0.shape[1], dtype=torch.float64, device=device)
    t = 1.0

    # define storage
    if save_path:
        path_all = [xt]
        t_vec = [t]

    # forward Euler sampling
    for i in range(time_steps):
        # prior score evaluation
        alpha_t = alpha_fun(t)
        sigma2_t = sigma2_fun(t)


        # Evaluate the diffusion term
        diffuse = diffuse_fun(t)

        # Evaluate the drift term
        # drift = drift_fun(t)*xt - diffuse**2 * score_eval

        # Update
        if score_likelihood is not None:
#             zt = score_likelihood(xt, t)
#             print(zt.size())
            xt += -dt*( drift_fun(t)*xt+diffuse**2*((xt-alpha_t*x0)/sigma2_t)-\
                       diffuse**2*score_likelihood(idA_sub, xt, t, C)) +np.sqrt(dt)*diffuse*torch.randn_like(xt)
    
        else:
            xt += -dt*(drift_fun(t)*xt+diffuse**2*((xt-alpha_t*x0)/sigma2_t))+np.sqrt(dt)*diffuse*torch.randn_like(xt)
        
        xt = xt.to(torch.float64)
        # Store the state in the path
        if save_path:
            path_all.append(xt)
            t_vec.append(t)

        # update time
        t = t - dt

    if save_path:
        return path_all, t_vec
    else:
        return xt

In [ ]:
def systematic_resample_pytorch(weights):
    N = len(weights)
    
    # Generate positions with a consistent random offset
    positions = (torch.rand(1) + torch.arange(N)) / N

    # Compute cumulative sum of weights
    cumulative_sum = torch.cumsum(weights, dim=0)

    # Initialize the indexes tensor
    indexes = torch.zeros(N, dtype=torch.long)
    
    # Initialize pointers
    i, j = 0, 0
    while i < N and j < len(cumulative_sum):
        if positions[i] < cumulative_sum[j]:
            indexes[i] = j
            i += 1
        else:
            j += 1
    
    return indexes

## Run this block to obtain reference solution

In [ ]:
nx = 40
ny = 40
xa = 0 
xb = 2
yc = 0 
yd = 1
delta = 0.1
NPX = 2
NPY = 1
nt = 800
t0 = 0
T = 5
dy = (yd-yc) / ny

dt = (T-t0) / nt
nSub = NPX * NPY
device = 'cpu'
# torch.set_default_device('cpu')
subdom, hydraulic1d, len_edge_frac, diff_gamma, poros = \
            AdvDiff_DA.GenerateSubdom_AdvDiff(nx, ny, xa, xb, yc, yd,\
                                              delta, NPX, NPY, nt, t0, T, device)
RectMesh = [None]*2
n_ori_out = [None]*2
NumEms = [None]*2
NumDofus = [None]*2
NumIntEgs = [None]*2
NumBdEgs = [None]*2
IntEdge2dofu = [None]*2

# poros_frt = poros[2]*delta
for k in range(2):
    RectMesh[k] = subdom[k].RectMesh
    n_ori_out[k] = RectMesh[k].n_ori_out
    NumEms[k] = RectMesh[k].NumEms
    NumDofus[k] = RectMesh[k].NumDofus
    NumIntEgs[k] = RectMesh[k].NumIntEgs
    NumBdEgs[k] = RectMesh[k].NumBdEgs
    IntEdge2dofu[k] = RectMesh[k].IntEdge2dofu[:, 1:3]
    
StiffMat = AdvDiff_DA.Assemble_StiffMat_Darcy(subdom, ny, hydraulic1d, len_edge_frac)
Wf_decomp, Darcy_u = AdvDiff_DA.DarcySolver(RectMesh, StiffMat, nSub, ny)

poros_frt = poros[2]*delta
alp_frt = diff_gamma*delta

Dele = [None]*2
for k in range(2):
    Dele[k] = subdom[k].Dele
    
subdom = AdvDiff_DA.Rfrt_TransportSetting(subdom, nx, ny, NPX, NPY, nt, t0, T, dt, xa, xb, yc, yd,\
                                          len_edge_frac, hydraulic1d, diff_gamma, device)

StiffMat, Mat1d = AdvDiff_DA.Rfrt_Transport_StiffMat(subdom, nSub, nx, ny, dy, Darcy_u,\
                                                     dt, Dele, len_edge_frac, delta, alp_frt, poros_frt)

n_dim = n_ori_out[0]+n_ori_out[1]+3*ny+ny+1

X0 = torch.zeros((1, RectMesh[0].n_ori_out+RectMesh[1].n_ori_out+4*ny+1), dtype=torch.float64, device=device)
p0 = [None]*3
p0[0] = X0[0, RectMesh[0].NumDofus:(RectMesh[0].NumDofus+RectMesh[0].NumEms)]
p0[1] = X0[0, n_ori_out[0]+RectMesh[1].NumDofus:n_ori_out[0]+RectMesh[1].NumDofus+RectMesh[1].NumEms]
p0[2] = X0[0, n_ori_out[0]+n_ori_out[1]+2*ny:+n_ori_out[0]+n_ori_out[1]+2*ny+ny]

X0 = X0.to(device)
state_target = X0
# state_ref = torch.zeros(nt+1, n_dim, device = device)
# state_ref[[0], :] += state_target
# for i in range(nt):
#     print(i)
#     stateT = torch.transpose(state_ref[[i], :], 0, 1)
#     Sol  = AdvDiff_DA.Rfrt_Transport_Solver(p0, subdom, Mat1d, StiffMat, nSub, nx, ny,\
#                                             Darcy_u, dt, delta, poros_frt)
#     state_ref[[i+1], :] += torch.transpose(Sol[:, None], 0, 1)
    
#     p0[0] = Sol[RectMesh[0].NumDofus:(RectMesh[0].NumDofus+RectMesh[0].NumEms)]
#     p0[1] = Sol[n_ori_out[0]+RectMesh[1].NumDofus:n_ori_out[0]+RectMesh[1].NumDofus+RectMesh[1].NumEms]
#     p0[2] = Sol[n_ori_out[0]+n_ori_out[1]+2*ny:+n_ori_out[0]+n_ori_out[1]+2*ny+ny]

## Save the solution in .mat file

In [ ]:
state_save_ref = state_ref.clone()
state_ref_save = state_ref.cpu().numpy()

scipy.io.savemat('UnitedSF_RefSol_AdvDiff_v1.mat', {'ExactState': state_ref_save, 'nx':nx, 'ny':ny, 'nt':nt})

## United Filter's Setting

In [ ]:
TotalEms = NumEms[0]+NumEms[1]+ny
TotalEgs = NumDofus[0]+NumDofus[1]+2*ny
TotalLarg = NumIntEgs[0]+NumIntEgs[1]+NumBdEgs[0]+NumBdEgs[1]-ny+1

TotalEmsSubdom = NumEms[0]+NumEms[1]
TotalEgsSubdom = NumDofus[0]+NumDofus[1]

indices_A = torch.randperm(TotalEgsSubdom)[:int(0.5 * TotalEgsSubdom)]
indices_Afrt = torch.randperm(2*ny)[:int(0.5*2*ny)]+TotalEgsSubdom

indices_B = torch.randperm(TotalEmsSubdom)[:int(0.51 * TotalEmsSubdom)]+TotalEgs
indices_Bfrt = torch.randperm(ny)[:int(0.5*ny)]+TotalEgs+TotalEmsSubdom

indices_C = torch.randperm(TotalLarg)[:int(0.5 * TotalLarg)]+TotalEgs+TotalEms


scale_ind , _ = torch.sort(torch.cat([indices_A, indices_Afrt]))
scale_ind, _ = torch.sort(scale_ind)

#####
num_indices= indices_A.shape[0]+indices_Afrt.shape[0]+indices_B.shape[0]+indices_Bfrt.shape[0]+indices_C.shape[0]

# Combine and sort the selected indices
spa_indices, _ = torch.sort(torch.cat([indices_A, indices_Afrt, indices_B, indices_Bfrt, indices_C]))

length_indices = int(0.5*spa_indices.shape[0]) 
idA_sub = torch.randperm(spa_indices.shape[0])[:length_indices]  # Random permutation, then take the first num_indices
idA_sub, _ = torch.sort(idA_sub)

In [ ]:
data = scipy.io.loadmat('UnitedSF_RefSol_AdvDiff_v1.mat')
state_ref = torch.from_numpy(data['ExactState'])

ntEnSF = 50
filtering_steps = ntEnSF
timeTrue = torch.linspace(0, 1, nt+1)
tEnSF = torch.linspace(0, 1, filtering_steps+1)
indices = torch.searchsorted(timeTrue, tEnSF, right=False)

state_EnSF = state_ref[indices, :].clone()

state_EnSF_Swap = AdvDiff_DA.RearrangeSol(state_EnSF.clone(), RectMesh, ny)
dtEnSF = (T - t0) / ntEnSF
obs_sigma = 0.1
####################################################################
# EnSF setup
# define the diffusion process
eps_alpha = 0.05

# ensemble size
ensemble_size = 200
ensemble_true = 1
# forward Euler step
euler_steps = 200
sub_steps = 4

time_euler = torch.linspace(0, 1, euler_steps+1)
time_sub = torch.linspace(0, 1, sub_steps+1)
select_set = torch.searchsorted(time_euler, time_sub, right=False)

# device = 'cpu'

def g_tau(t):
    return 1-t

# saving file name
exp_name = 'JointStatePara_RFrtModel_AdvDiff_Noise0001'
# exp_name = 'JointStatePara_RFrtModel_AdvDiff_Noise01'
x_state = 5*torch.randn(ensemble_size, n_dim, dtype=torch.float64, device=device)
# x_state = x_state.to(torch.float16)
mem_state = state_target.element_size() * state_target.nelement()/1e+6
mem_ensemble = mem_state * ensemble_size
print(f'single state memory: {mem_state:.2f} MB')
print(f'state ensemble memory: {mem_ensemble:.2f} MB')

rmse_all = []
obs_save = []
est_save = torch.zeros(filtering_steps+1, n_dim, device = device)
est_save[[0], :] += torch.mean(x_state,dim=0)

x0filter = x_state
theta0 = torch.tensor([600, 600, 400], dtype=torch.float64, device =device); #Initial guess
Cov = torch.eye(3, dtype=torch.float64, device =device) * torch.tensor([800, 800, 400], dtype=torch.float64, device =device)

n = 40                     # Number of particles considered     
l= 4

particles =torch.distributions.MultivariateNormal(theta0, Cov).sample((n,))
xs = [theta0]

weights = torch.ones(n, dtype=torch.float64, device=device)*(1/n)
Para = torch.sum(particles * weights.unsqueeze(1), dim=0) 

X = torch.zeros((ntEnSF+1, n_dim), dtype=torch.float64, device=device) #545 is the dof in the mesh considered
X[0, :] += state_EnSF[0, :]
for i in range(1, ntEnSF+1):
    X[i, :] += state_EnSF[i, :]

In [ ]:
state_United = state_EnSF_Swap[:, spa_indices].clone()
# SDE_Sigma = 0.001
# SDE_Sigma = 0.1

DE_Sigma = 0.00001
SDE_Sigma = 0.001
state_United = state_United.to(torch.float64)
factorR = 0.2
# R = factorR*torch.eye(n_dim, dtype=torch.float64, device=device)   # Covariance considered 
# factor = 100
# R_scaled = factor * R

## Run the United Filter iterations

In [ ]:
for i in range(filtering_steps):
    print(f'step={i}:')
    t1 = time.time()
    obs = state_United[[i+1], :].clone()
    state_United_Scale = state_United[[i+1], :].clone()
    
    ## Scale the observations and the prediction before running the EnSF
    ind_scale1 = torch.nonzero(((-1e-20 <= obs) & (obs < 0)) | ((0 <= obs) & (obs < 1e-20)), as_tuple= True)[1]
    ind_scale2 = torch.nonzero(((-1e-19 <= obs) & (obs < -1e-20)) | ((1e-20 <= obs) & (obs < 1e-19)), as_tuple= True)[1]
    ind_scale3 = torch.nonzero(((-1e-18 <= obs) & (obs < -1e-19)) | ((1e-19 <= obs) & (obs < 1e-18)), as_tuple= True)[1]
    ind_scale4 = torch.nonzero(((-1e-17 <= obs) & (obs < -1e-18)) | ((1e-18 <= obs) & (obs < 1e-17)), as_tuple= True)[1]
    ind_scale5 = torch.nonzero(((-1e-16 <= obs) & (obs < -1e-17)) | ((1e-17 <= obs) & (obs < 1e-16)), as_tuple= True)[1]
    ind_scale6 = torch.nonzero(((-1e-15 <= obs) & (obs < -1e-16)) | ((1e-16 <= obs) & (obs < 1e-15)), as_tuple= True)[1]
    ind_scale7 = torch.nonzero(((-1e-14 <= obs) & (obs < -1e-15)) | ((1e-15 <= obs) & (obs < 1e-14)), as_tuple= True)[1]
    ind_scale8 = torch.nonzero(((-1e-13 <= obs) & (obs < -1e-14)) | ((1e-14 <= obs) & (obs < 1e-13)), as_tuple= True)[1]
    ind_scale9 = torch.nonzero(((-1e-12 <= obs) & (obs < -1e-13)) | ((1e-13 <= obs) & (obs < 1e-12)), as_tuple= True)[1]
    ind_scale10 = torch.nonzero(((-1e-11 <= obs) & (obs < -1e-12)) | ((1e-12 <= obs) & (obs < 1e-11)), as_tuple= True)[1]
    ind_scale11 = torch.nonzero(((-1e-10 <= obs) & (obs < -1e-11)) | ((1e-11 <= obs) & (obs < 1e-10)), as_tuple= True)[1]
    ind_scale12 = torch.nonzero(((-1e-9 <= obs) & (obs < -1e-10)) | ((1e-10 <= obs) & (obs < 1e-9)), as_tuple= True)[1]
    ind_scale13 = torch.nonzero(((-1e-8 <= obs) & (obs < -1e-9)) | ((1e-9 <= obs) & (obs < 1e-8)), as_tuple= True)[1]
    ind_scale14 = torch.nonzero(((-1e-7 <= obs) & (obs < -1e-8)) | ((1e-8 <= obs) & (obs < 1e-7)), as_tuple= True)[1]
    ind_scale15 = torch.nonzero(((-1e-6 <= obs) & (obs < -1e-7)) | ((1e-7 <= obs) & (obs < 1e-6)), as_tuple= True)[1]
    ind_scale16 = torch.nonzero(((-1e-5 <= obs) & (obs < -1e-6)) | ((1e-6 <= obs) & (obs < 1e-5)), as_tuple= True)[1]
    ind_scale17 = torch.nonzero(((-1e-4 <= obs) & (obs < -1e-5)) | ((1e-5 <= obs) & (obs < 1e-4)), as_tuple= True)[1]
    ind_scale18 = torch.nonzero(((-1e-3 <= obs) & (obs < -1e-4)) | ((1e-4 <= obs) & (obs < 1e-3)), as_tuple= True)[1]
    ind_scale19 = torch.nonzero(((-1e-2 <= obs) & (obs < -1e-3)) | ((1e-3 <= obs) & (obs < 1e-2)), as_tuple= True)[1]
    
    
    obs[:, ind_scale1] *= 1e+20
    obs[:, ind_scale2] *= 1e+19
    obs[:, ind_scale3] *= 1e+18
    obs[:, ind_scale4] *= 1e+17
    obs[:, ind_scale5] *= 1e+16
    obs[:, ind_scale6] *= 1e+15
    obs[:, ind_scale7] *= 1e+14
    obs[:, ind_scale8] *= 1e+13
    obs[:, ind_scale9] *= 1e+12
    obs[:, ind_scale10] *= 1e+11
    obs[:, ind_scale11] *= 1e+10
    obs[:, ind_scale12] *= 1e+9
    obs[:, ind_scale13] *= 1e+8
    obs[:, ind_scale14] *= 1e+7
    obs[:, ind_scale15] *= 1e+6
    obs[:, ind_scale16] *= 1e+5
    obs[:, ind_scale17] *= 1e+4
    obs[:, ind_scale18] *= 1e+3
    obs[:, ind_scale19] *= 1e+2

    state_United_Scale[:, ind_scale1] *= 1e+20
    state_United_Scale[:, ind_scale2] *= 1e+19
    state_United_Scale[:, ind_scale3] *= 1e+18
    state_United_Scale[:, ind_scale4] *= 1e+17
    state_United_Scale[:, ind_scale5] *= 1e+16
    state_United_Scale[:, ind_scale6] *= 1e+15
    state_United_Scale[:, ind_scale7] *= 1e+14
    state_United_Scale[:, ind_scale8] *= 1e+13
    state_United_Scale[:, ind_scale9] *= 1e+12
    state_United_Scale[:, ind_scale10] *= 1e+11
    state_United_Scale[:, ind_scale11] *= 1e+10
    state_United_Scale[:, ind_scale12] *= 1e+9
    state_United_Scale[:, ind_scale13] *= 1e+8
    state_United_Scale[:, ind_scale14] *= 1e+7
    state_United_Scale[:, ind_scale15] *= 1e+6
    state_United_Scale[:, ind_scale16] *= 1e+5
    state_United_Scale[:, ind_scale17] *= 1e+4
    state_United_Scale[:, ind_scale18] *= 1e+3
    state_United_Scale[:, ind_scale19] *= 1e+2
    
    obs[:, idA_sub] = torch.atan(state_United_Scale[:, idA_sub].clone())
    obs += torch.randn_like(state_United[[i+1], :])*obs_sigma
    def score_likelihood(idA_sub, xt, t, C):
        # obs: (d)
        # xt: (ensemble, d)
        A = -(xt-obs)/obs_sigma**2

        score_x = A.clone()
        score_x[:, idA_sub] = -(torch.atan(xt[:, idA_sub]) - obs[:, idA_sub])/obs_sigma**2 * (1./(1+xt[:, idA_sub]**2))
        tau = g_tau(t)
        return tau*score_x/C
    
    x0EnSF = x_state.clone()
    x0filterT = torch.transpose(x0EnSF.clone(), 0, 1)
    p0Loop = [None]*3
    p0Loop[0] = x0filterT[RectMesh[0].NumDofus:(RectMesh[0].NumDofus+RectMesh[0].NumEms), :]
    p0Loop[1] = x0filterT[n_ori_out[0]+RectMesh[1].NumDofus:n_ori_out[0]+RectMesh[1].NumDofus+RectMesh[1].NumEms, :]
    p0Loop[2] = x0filterT[n_ori_out[0]+n_ori_out[1]+2*ny:+n_ori_out[0]+n_ori_out[1]+2*ny+ny, :]

    Darcy_u_perb = [None]*3
    for k in range(3):
        Darcy_u_perb[k] = Darcy_u[k].clone()+0.00005*torch.randn_like(Darcy_u[k], dtype=torch.float64)
        
    for j in range(l):
        weights = torch.ones(n, dtype=torch.float64, device=device)*(1/n)
        Para_Start = Para.clone()
    
        ##### EnSF 
        Solver_t1 = time.time()
        DeleLoop = [None]*2
        for k in range(2):
            DeleLoop[k] = 1 / Para_Start[k]
            
        MatLoop, Mat1dLoop = AdvDiff_DA.Rfrt_Transport_StiffMat(subdom, nSub, nx, ny, dy, Darcy_u_perb, \
                                                                dtEnSF, DeleLoop, len_edge_frac, delta,\
                                                                1 / Para_Start[2], poros_frt)
        
        sln_state = AdvDiff_DA.Rfrt_Transport_Solver(p0Loop, subdom, Mat1dLoop, MatLoop, nSub, nx, ny, \
                                                     Darcy_u_perb, dtEnSF, delta, poros_frt)

        Solver_t2 = time.time()
        # x_state = torch.transpose(sln_state, 0, 1)+\
        #             np.sqrt(dtEnSF)*SDE_Sigma*torch.randn_like(x_state, dtype=torch.float64, device=device)
        
        # print(RectMesh[0])
        x_state = AdvDiff_DA.RearrangeSol(x_state, RectMesh, ny)

        noise_Pres = \
            np.sqrt(dtEnSF)*SDE_Sigma_Pres*torch.randn_like(x_state[:, TotalEgs+torch.arange(0, TotalEms)],\
                                                        dtype=torch.float64, device=device)
        noise_Vel = \
            np.sqrt(dtEnSF)*SDE_Sigma_Vel*torch.randn_like(x_state[:, :TotalEgs], dtype=torch.float64, device=device)
    
        noise_Larg = \
            0*np.sqrt(dtEnSF)*SDE_Sigma_Vel*torch.randn_like(x_state[:, TotalEgs+TotalEms+torch.arange(0, TotalLarg)], \
                                                                dtype=torch.float64, device=device)

        noise = torch.cat((noise_Vel, noise_Pres, noise_Larg), dim=1)
        x_state += noise
        x_state_clone = x_state[:, spa_indices].clone()
        
        # ind_xscale1 = torch.nonzero((-1e-20 <= x_state_clone) & (x_state_clone < 1e-20), as_tuple = True)[1]
        # ind_xscale2 = torch.nonzero(((-1e-19 <= x_state_clone) & (x_state_clone < -1e-20)) | \
        #                             ((1e-20 <= x_state_clone) & (x_state_clone < 1e-19)), as_tuple= True)[1]
        
        # ind_xscale3 = torch.nonzero(((-1e-18 <= x_state_clone) & (x_state_clone < -1e-19)) | \
        #                             ((1e-19 <= x_state_clone) & (x_state_clone < 1e-18)), as_tuple= True)[1]
        # ind_xscale4 = torch.nonzero(((-1e-17 <= x_state_clone) & (x_state_clone < -1e-18)) |\
        #                             ((1e-18 <= x_state_clone) & (x_state_clone< 1e-17)), as_tuple= True)[1]
        
        # ind_xscale5 = torch.nonzero(((-1e-16 <= x_state_clone) & (x_state_clone < -1e-17)) | \
        #                             ((1e-17 <= x_state_clone) & (x_state_clone< 1e-16)), as_tuple= True)[1]
        # ind_xscale6 = torch.nonzero(((-1e-15 <= x_state_clone) & (x_state_clone < -1e-16)) | \
        #                             ((1e-16 <= x_state_clone) & (x_state_clone < 1e-15)), as_tuple= True)[1]
        # ind_xscale7 = torch.nonzero(((-1e-14 <= x_state_clone) & (x_state_clone < -1e-15)) | \
        #                             ((1e-15 <= x_state_clone) & (x_state_clone < 1e-14)), as_tuple= True)[1]
        # ind_xscale8 = torch.nonzero(((-1e-13 <= x_state_clone) & (x_state_clone < -1e-14)) |\
        #                             ((1e-14 <= x_state_clone) & (x_state_clone < 1e-13)), as_tuple= True)[1]
        # ind_xscale9 = torch.nonzero(((-1e-12 <= x_state_clone) & (x_state_clone < -1e-13)) | \
        #                             ((1e-13 <= x_state_clone) & (x_state_clone < 1e-12)), as_tuple= True)[1]
        # ind_xscale10 = torch.nonzero(((-1e-11 <= x_state_clone) & (x_state_clone < -1e-12)) | \
        #                              ((1e-12 <= x_state_clone) & (x_state_clone < 1e-11)), as_tuple= True)[1]
        # ind_xscale11 = torch.nonzero(((-1e-10 <= x_state_clone) & (x_state_clone < -1e-11)) | \
        #                              ((1e-11 <= x_state_clone) & (x_state_clone < 1e-10)), as_tuple= True)[1]
        # ind_xscale12 = torch.nonzero(((-1e-9 <= x_state_clone) & (x_state_clone < -1e-10)) | \
        #                              ((1e-10 <= x_state_clone) & (x_state_clone < 1e-9)), as_tuple= True)[1]
        # ind_xscale13 = torch.nonzero(((-1e-8 <= x_state_clone) & (x_state_clone < -1e-9)) | \
        #                              ((1e-9 <= x_state_clone) & (x_state_clone < 1e-8)), as_tuple= True)[1]
        # ind_xscale14 = torch.nonzero(((-1e-7 <= x_state_clone) & (x_state_clone < -1e-8)) | \
        #                              ((1e-8 <= x_state_clone) & (x_state_clone < 1e-7)), as_tuple= True)[1]
        # ind_xscale15 = torch.nonzero(((-1e-6 <= x_state_clone) & (x_state_clone < -1e-7)) | \
        #                              ((1e-7 <= x_state_clone) & (x_state_clone < 1e-6)), as_tuple= True)[1]
        # ind_xscale16 = torch.nonzero(((-1e-5 <= x_state_clone) & (x_state_clone < -1e-6)) | \
        #                              ((1e-6 <= x_state_clone) & (x_state_clone < 1e-5)), as_tuple= True)[1]
        # ind_xscale17 = torch.nonzero(((-1e-4 <= x_state_clone) & (x_state_clone < -1e-5)) | \
        #                              ((1e-5 <= x_state_clone) & (x_state_clone < 1e-4)), as_tuple= True)[1]
        # ind_xscale18 = torch.nonzero(((-1e-3 <= x_state_clone) & (x_state_clone < -1e-4)) | \
        #                              ((1e-4 <= x_state_clone) & (x_state_clone < 1e-3)), as_tuple= True)[1]
        # ind_xscale19 = torch.nonzero(((-1e-2 <= x_state_clone) & (x_state_clone < -1e-3)) | \
        #                              ((1e-3 <= x_state_clone) & (x_state_clone < 1e-2)), as_tuple= True)[1]
        
        ind_xscale1 = torch.nonzero(((-1e-20 <= x_state_clone) & (x_state_clone < 0)) | \
                                    ((0 <= x_state_clone) & (x_state_clone < 1e-20)), as_tuple= False)
        
        ind_xscale2 = torch.nonzero(((-1e-19 <= x_state_clone) & (x_state_clone < -1e-20)) | \
                                    ((1e-20 <= x_state_clone) & (x_state_clone < 1e-19)), as_tuple= False)
        
        ind_xscale3 = torch.nonzero(((-1e-18 <= x_state_clone) & (x_state_clone < -1e-19)) | \
                                    ((1e-19 <= x_state_clone) & (x_state_clone < 1e-18)), as_tuple= False)
        ind_xscale4 = torch.nonzero(((-1e-17 <= x_state_clone) & (x_state_clone < -1e-18)) |\
                                    ((1e-18 <= x_state_clone) & (x_state_clone< 1e-17)), as_tuple= False)
        
        ind_xscale5 = torch.nonzero(((-1e-16 <= x_state_clone) & (x_state_clone < -1e-17)) | \
                                    ((1e-17 <= x_state_clone) & (x_state_clone< 1e-16)), as_tuple= False)
        ind_xscale6 = torch.nonzero(((-1e-15 <= x_state_clone) & (x_state_clone < -1e-16)) | \
                                    ((1e-16 <= x_state_clone) & (x_state_clone < 1e-15)), as_tuple= False)
        ind_xscale7 = torch.nonzero(((-1e-14 <= x_state_clone) & (x_state_clone < -1e-15)) | \
                                    ((1e-15 <= x_state_clone) & (x_state_clone < 1e-14)), as_tuple= False)
        ind_xscale8 = torch.nonzero(((-1e-13 <= x_state_clone) & (x_state_clone < -1e-14)) |\
                                    ((1e-14 <= x_state_clone) & (x_state_clone < 1e-13)), as_tuple= False)
        ind_xscale9 = torch.nonzero(((-1e-12 <= x_state_clone) & (x_state_clone < -1e-13)) | \
                                    ((1e-13 <= x_state_clone) & (x_state_clone < 1e-12)), as_tuple= False)
        ind_xscale10 = torch.nonzero(((-1e-11 <= x_state_clone) & (x_state_clone < -1e-12)) | \
                                     ((1e-12 <= x_state_clone) & (x_state_clone < 1e-11)), as_tuple= False)
        ind_xscale11 = torch.nonzero(((-1e-10 <= x_state_clone) & (x_state_clone < -1e-11)) | \
                                     ((1e-11 <= x_state_clone) & (x_state_clone < 1e-10)), as_tuple= False)
        ind_xscale12 = torch.nonzero(((-1e-9 <= x_state_clone) & (x_state_clone < -1e-10)) | \
                                     ((1e-10 <= x_state_clone) & (x_state_clone < 1e-9)), as_tuple= False)
        ind_xscale13 = torch.nonzero(((-1e-8 <= x_state_clone) & (x_state_clone < -1e-9)) | \
                                     ((1e-9 <= x_state_clone) & (x_state_clone < 1e-8)), as_tuple= False)
        ind_xscale14 = torch.nonzero(((-1e-7 <= x_state_clone) & (x_state_clone < -1e-8)) | \
                                     ((1e-8 <= x_state_clone) & (x_state_clone < 1e-7)), as_tuple= False)
        ind_xscale15 = torch.nonzero(((-1e-6 <= x_state_clone) & (x_state_clone < -1e-7)) | \
                                     ((1e-7 <= x_state_clone) & (x_state_clone < 1e-6)), as_tuple= False)
        ind_xscale16 = torch.nonzero(((-1e-5 <= x_state_clone) & (x_state_clone < -1e-6)) | \
                                     ((1e-6 <= x_state_clone) & (x_state_clone < 1e-5)), as_tuple= False)
        ind_xscale17 = torch.nonzero(((-1e-4 <= x_state_clone) & (x_state_clone < -1e-5)) | \
                                     ((1e-5 <= x_state_clone) & (x_state_clone < 1e-4)), as_tuple= False)
        ind_xscale18 = torch.nonzero(((-1e-3 <= x_state_clone) & (x_state_clone < -1e-4)) | \
                                     ((1e-4 <= x_state_clone) & (x_state_clone < 1e-3)), as_tuple= False)
        ind_xscale19 = torch.nonzero(((-1e-2 <= x_state_clone) & (x_state_clone < -1e-3)) | \
                                     ((1e-3 <= x_state_clone) & (x_state_clone < 1e-2)), as_tuple= False)
        
        x_state_clone[ind_xscale1[:, 0], ind_xscale1[:, 1]] *= 1e+20
        x_state_clone[ind_xscale2[:, 0], ind_xscale2[:, 1]] *= 1e+19
        x_state_clone[ind_xscale3[:, 0], ind_xscale3[:, 1]] *= 1e+18
        x_state_clone[ind_xscale4[:, 0], ind_xscale4[:, 1]] *= 1e+17
        x_state_clone[ind_xscale5[:, 0], ind_xscale5[:, 1]] *= 1e+16
        x_state_clone[ind_xscale6[:, 0], ind_xscale6[:, 1]] *= 1e+15
        x_state_clone[ind_xscale7[:, 0], ind_xscale7[:, 1]] *= 1e+14
        x_state_clone[ind_xscale8[:, 0], ind_xscale8[:, 1]] *= 1e+13
        x_state_clone[ind_xscale9[:, 0], ind_xscale9[:, 1]] *= 1e+12
        x_state_clone[ind_xscale10[:, 0], ind_xscale10[:, 1]] *= 1e+11
        x_state_clone[ind_xscale11[:, 0], ind_xscale11[:, 1]] *= 1e+10
        x_state_clone[ind_xscale12[:, 0], ind_xscale12[:, 1]] *= 1e+9
        x_state_clone[ind_xscale13[:, 0], ind_xscale13[:, 1]] *= 1e+8
        x_state_clone[ind_xscale14[:, 0], ind_xscale14[:, 1]] *= 1e+7
        x_state_clone[ind_xscale15[:, 0], ind_xscale15[:, 1]] *= 1e+6
        x_state_clone[ind_xscale16[:, 0], ind_xscale16[:, 1]] *= 1e+5
        x_state_clone[ind_xscale17[:, 0], ind_xscale17[:, 1]] *= 1e+4
        x_state_clone[ind_xscale18[:, 0], ind_xscale18[:, 1]] *= 1e+3
        x_state_clone[ind_xscale19[:, 0], ind_xscale19[:, 1]] *= 1e+2
        
        # x_state_clone[:, ind_xscale1] *= 1e+20
        # x_state_clone[:, ind_xscale2] *= 1e+19
        # x_state_clone[:, ind_xscale3] *= 1e+18
        # x_state_clone[:, ind_xscale4] *= 1e+17
        # x_state_clone[:, ind_xscale5] *= 1e+16
        # x_state_clone[:, ind_xscale6] *= 1e+15
        # x_state_clone[:, ind_xscale7] *= 1e+14
        # x_state_clone[:, ind_xscale8] *= 1e+13
        # x_state_clone[:, ind_xscale9] *= 1e+12
        # x_state_clone[:, ind_xscale10] *= 1e+11
        # x_state_clone[:, ind_xscale11] *= 1e+10
        # x_state_clone[:, ind_xscale12] *= 1e+9
        # x_state_clone[:, ind_xscale13] *= 1e+8
        # x_state_clone[:, ind_xscale14] *= 1e+7
        # x_state_clone[:, ind_xscale15] *= 1e+6
        # x_state_clone[:, ind_xscale16] *= 1e+5
        # x_state_clone[:, ind_xscale17] *= 1e+4
        # x_state_clone[:, ind_xscale18] *= 1e+3
        # x_state_clone[:, ind_xscale19] *= 1e+2

        EnSF_t1 = time.time()
        sln_bar = reverse_SDE(num_indices, select_set, idA_sub, x0=x_state_clone, \
                              time_steps=euler_steps, C=1,  score_likelihood=score_likelihood)

        EnSF_t2 = time.time()
        
        sln_bar[:, ind_scale1] /= 1e+20
        sln_bar[:, ind_scale2] /= 1e+19
        sln_bar[:, ind_scale3] /= 1e+18
        sln_bar[:, ind_scale4] /= 1e+17
        sln_bar[:, ind_scale5] /= 1e+16
        sln_bar[:, ind_scale6] /= 1e+15
        sln_bar[:, ind_scale7] /= 1e+14
        sln_bar[:, ind_scale8] /= 1e+13
        sln_bar[:, ind_scale9] /= 1e+12
        sln_bar[:, ind_scale10] /= 1e+11
        sln_bar[:, ind_scale11] /= 1e+10
        sln_bar[:, ind_scale12] /= 1e+9
        sln_bar[:, ind_scale13] /= 1e+8
        sln_bar[:, ind_scale14] /= 1e+7
        sln_bar[:, ind_scale15] /= 1e+6
        sln_bar[:, ind_scale16] /= 1e+5
        sln_bar[:, ind_scale17] /= 1e+4
        sln_bar[:, ind_scale18] /= 1e+3
        sln_bar[:, ind_scale19] /= 1e+2
        
        # sln_bar[:, ind_xscale1] /= 1e+20
        # sln_bar[:, ind_xscale2] /= 1e+19
        # sln_bar[:, ind_xscale3] /= 1e+18
        # sln_bar[:, ind_xscale4] /= 1e+17
        # sln_bar[:, ind_xscale5] /= 1e+16
        # sln_bar[:, ind_xscale6] /= 1e+15
        # sln_bar[:, ind_xscale7] /= 1e+14
        # sln_bar[:, ind_xscale8] /= 1e+13
        # sln_bar[:, ind_xscale9] /= 1e+12
        # sln_bar[:, ind_xscale10] /= 1e+11
        # sln_bar[:, ind_xscale11] /= 1e+10
        # sln_bar[:, ind_xscale12] /= 1e+9
        # sln_bar[:, ind_xscale13] /= 1e+8
        # sln_bar[:, ind_xscale14] /= 1e+7
        # sln_bar[:, ind_xscale15] /= 1e+6
        # sln_bar[:, ind_xscale16] /= 1e+5
        # sln_bar[:, ind_xscale17] /= 1e+4
        # sln_bar[:, ind_xscale18] /= 1e+3
        # sln_bar[:, ind_xscale19] /= 1e+2
        
        x_state[:, spa_indices] = sln_bar.clone()
        
        x_state[:, TotalEgs + torch.arange(0, NumEms[0])] =\
            torch.clamp(x_state[:, TotalEgs + torch.arange(0, NumEms[0])],\
                        min=0, max = 2.999999)

        x_state[:, TotalEgs+NumEms[0]+torch.arange(0, NumEms[1])] =\
                    torch.clamp(x_state[:, TotalEgs+NumEms[0]+torch.arange(0, NumEms[1])],\
                                min=0, max = 2.999999)

        x_state[:, TotalEgs+NumEms[0]+NumEms[1]+torch.arange(0, ny)] =\
                    torch.clamp(x_state[:, TotalEgs+NumEms[0]+NumEms[1]+torch.arange(0, ny)],\
                                min=0, max = 3)
        
        x_state[:, TotalEgs+TotalEms:n_dim] =\
                    torch.clamp(x_state[:, TotalEgs+TotalEms:n_dim], min=0, max = 3)
               
        x_state[:, :NumDofus[0]+NumDofus[1]] =\
            torch.clamp(x_state[:, :NumDofus[0]+NumDofus[1]], min=-0.00008, max = 0.00008)

#         x_state[:, NumDofus[0]+NumDofus[1]+torch.arange(0, 2*ny)] =\
#                 torch.clamp(x_state[:, NumDofus[0]+NumDofus[1]+torch.arange(0, 2*ny)], min=-0.009, max = 0.009)
        
        x_state[:, TotalEgs+TotalEms+NumIntEgs[0]+ny+torch.arange(0, ny)] = 3
        x_state[:, TotalEgs+TotalEms+NumIntEgs[0]+ny+ny+torch.arange(0, ny)] = 0

        x_state[:, TotalEgs+TotalEms+NumIntEgs[0]+NumBdEgs[0]-ny+NumIntEgs[1]+ny+torch.arange(0, ny)] = 3
    
        x_state[:, TotalEgs+TotalEms+NumIntEgs[0]+NumBdEgs[0]-ny+NumIntEgs[1]+ny+ny+torch.arange(0, ny)] = 0

        x_state[:, TotalEgs+TotalEms+NumIntEgs[0]+NumBdEgs[0]-ny+NumIntEgs[1]+NumBdEgs[1]-ny+ny-1] = 3
    
        x_state[:, TotalEgs+TotalEms+NumIntEgs[0]+NumBdEgs[0]-ny+NumIntEgs[1]+NumBdEgs[1]-ny+ny] = 0
        
        # average_values1 = (x_state[:, IntEdge2dofu[0][:, 0]]-x_state[:, IntEdge2dofu[0][:, 1]])/2
        # average_values2 = (x_state[:, NumDofus[0]+IntEdge2dofu[1][:, 0]]-x_state[:, NumDofus[0]+IntEdge2dofu[1][:, 1]])/2

        # x_statefrt = x_state[:, NumDofus[0]+NumDofus[1]+torch.arange(0, 2*ny)]
        # x_state_mid = x_statefrt[:, 1:-1]
        # average_value3 = (x_state_mid[:, 0::2]-x_state_mid[:, 1::2])/2
        # odd_indices = torch.arange(2, 2*ny-1, step=2)
        # even_indices = torch.arange(1, 2*ny-1, step=2)

        # x_state[:, NumDofus[0]+NumDofus[1]+even_indices] = average_value3
        # x_state[:, NumDofus[0]+NumDofus[1]+odd_indices] = (-1)*average_value3
        
        # x_state[:, IntEdge2dofu[0][:, 0]] = average_values1
        # x_state[:, IntEdge2dofu[0][:, 1]] = (-1)*average_values1
        
        # x_state[:, NumDofus[0]+IntEdge2dofu[1][:, 0]] = average_values2
        # x_state[:, NumDofus[0]+IntEdge2dofu[1][:, 1]] = (-1)*average_values2

        
        x_state[:, RectMesh[0].bd2dofu[subdom[0].neumann_bd_transport]] = 0
        x_state[:, NumDofus[0]+RectMesh[1].bd2dofu[subdom[1].neumann_bd_transport]] = 0

        x_state[:, RectMesh[0].bd2dofu[ny+ny+torch.arange(0, nx)]] = \
                torch.where(x_state[:, RectMesh[0].bd2dofu[ny+ny+torch.arange(0, nx)]] < 0, \
                            x_state[:, RectMesh[0].bd2dofu[ny+ny+torch.arange(0, nx)]], \
                            (-1)*x_state[:, RectMesh[0].bd2dofu[ny+ny+torch.arange(0, nx)]])

        x_state[:, NumDofus[0]+RectMesh[1].bd2dofu[ny+ny+torch.arange(0, nx)]] = \
                torch.where(x_state[:, NumDofus[0]+RectMesh[0].bd2dofu[ny+ny+torch.arange(0, nx)]] < 0, \
                            x_state[:, NumDofus[0]+RectMesh[0].bd2dofu[ny+ny+torch.arange(0, nx)]], \
                            (-1)*x_state[:, NumDofus[0]+RectMesh[0].bd2dofu[ny+ny+torch.arange(0, nx)]])

        # x_state[:, NumDofus[0]+NumDofus[1]] = \
        #         torch.where(x_state[:, NumDofus[0]+NumDofus[1]] < 0, \
        #                     x_state[:, NumDofus[0]+NumDofus[1]], \
        #                     (-1)*x_state[:, NumDofus[0]+NumDofus[1]])
        
        x_state = AdvDiff_DA.RearrangeSol_Reverse(x_state, RectMesh, ny)
        ####     
        xstate_bar = torch.mean(x_state,dim=0) # \bar{X}^{(l)}_{i+1}
                
#         factor1 = torch.tensor([2800, 2800, 800], dtype=torch.float64)
#         factor2 = torch.tensor([3000, 3000, 970], dtype=torch.float64)
#         Ep = torch.diag(torch.tensor([18000, 18000, 6400], dtype=torch.float64))

#         # replacement_values1 = torch.tensor([3200, 4900, 1000], dtype=torch.float64)  # Example replacement values #v2
#         replacement_values1 = torch.tensor([4900, 4900, 2500], dtype=torch.float64)  # Example replacement values 
#         replacement_values2 = torch.tensor([0.0000005, 0.0000005, 0.00001], dtype=torch.float64)
    
#         for jj in range(len(Para_Start)):
#             if Para_Start[jj] > factor1[jj] and Para_Start[jj] < factor2[jj]:  # Condition to check
#                 print(f"Replacing diagonal entry {jj} of Ep by 1st value")
#                 Ep[jj, jj] = replacement_values1[jj]  # Replace the diagonal entry 
#             elif Para_Start[jj] > factor2[jj]:  # Condition to check
#                 print(f"Replacing diagonal entry {jj} of Ep by 2nd value")
#                 Ep[jj, jj] = replacement_values2[jj]
        
        if i <= int(5*filtering_steps // 50):
            Ep = torch.diag(torch.tensor([18000, 18000, 4000], dtype=torch.float64))  
        elif i < int(15*filtering_steps // 50) and i > int(5*filtering_steps // 50):
            Ep = torch.diag(torch.tensor([200, 200, 400], dtype=torch.float64))            
        else:
            Ep = torch.diag(torch.tensor([0.01, 0.01, 0.1], dtype=torch.float64))
        # Check if i is in the desired range
#     # Replace diagonal values based on IntmediaPara_Start
        cholEp = torch.linalg.cholesky(Ep)
        particles += torch.randn(n, 3, dtype=torch.float64)@cholEp #\tilde{\gamma}^{k, (l+1)}_{i+1}
        
        for qq in range(particles.shape[1]):  # Iterate over columns
            condition = particles[:, qq] < 0  # Check for negative values in the column
            if condition.any():  # If there are negative values
                print(qq)
                print(f"negative detected in {'d2' if qq == 1 else 'd1' if qq == 0 else 'd3'}")
                # Find the smallest positive entry in the column
                positive_values = particles[:, qq][particles[:, qq] > 0]
                if positive_values.numel() > 0:  # Ensure there are positive values
                    a = positive_values.min()
                    smallest_positive = a+100*abs(torch.randn_like(a, dtype=torch.float64))
                    # Replace negative values with the smallest positive entry
                    particles[:, qq] = torch.where(
                        condition,
                        smallest_positive,
                        particles[:, qq]
                    )  
        
        z = xstate_bar.clone() # 
        P = torch.zeros(n, n_dim, dtype=torch.float64, device=device)

        x0DFT = torch.transpose(est_save[[i], :].clone(), 0, 1)
        p0DF = [None]*3
        p0DF[0] = x0DFT[RectMesh[0].NumDofus:(RectMesh[0].NumDofus+RectMesh[0].NumEms), :].clone()
        p0DF[1] = x0DFT[n_ori_out[0]+RectMesh[1].NumDofus:n_ori_out[0]+RectMesh[1].NumDofus+RectMesh[1].NumEms, :].clone()
        p0DF[2] = x0DFT[n_ori_out[0]+n_ori_out[1]+2*ny:+n_ori_out[0]+n_ori_out[1]+2*ny+ny, :].clone()
        
        p0DF[0] = p0DF[0].to(torch.float64)
        p0DF[1] = p0DF[1].to(torch.float64)
        p0DF[2] = p0DF[2].to(torch.float64)

        DF_t1 = time.time()
        for o, particle in enumerate(particles):            
            DeleDF = [None]*2
            for k in range(2):
                DeleDF[k] = 1 / particle[k]

            MatDF, Mat1dDF = AdvDiff_DA.Rfrt_Transport_StiffMat(subdom, nSub, nx, ny, dy, Darcy_u_perb, \
                                                                dtEnSF, DeleDF, len_edge_frac, delta,\
                                                                1 / particle[2],  poros_frt)
                    
            SlnDF = AdvDiff_DA.Rfrt_Transport_Solver(p0DF, subdom, Mat1dDF, MatDF, nSub, nx, ny,\
                                                     Darcy_u_perb, dtEnSF, delta, poros_frt)
            SlnDFT = torch.transpose(SlnDF[:, None], 0, 1)
            P[o, :] += SlnDFT[-1, :] # \bar{X}^{\tilde{\gamma}^{k, (l+1)}_{i+1}}_{i+1}

        P -=  z.unsqueeze(0)   # Broadcasting is already efficient

        # Step 2: Solve R * X = P^T for X
        Rsolved = P*(1/factorR)
        # Step 3: Compute norms
        Rnorm = torch.sum(Rsolved * P, dim=1)
        Rnorm_clone = Rnorm / 1        
        
        if torch.min(Rnorm_clone) > 1000:
            alp_factor = torch.min(Rnorm_clone) / 1000 
#    
            Rnorm_clone /= alp_factor
            
   

        # Step 4: Update weights
        weights.mul_(torch.exp(-0.5 * Rnorm_clone))  # In-place operation
        # print(weights)
        weights.add_(1e-300)  # Add directly without creating new tensors
        weights.div_(weights.sum())  # Normalize in-place
     
        # Step 5: Resampling and updating particles
        indexes = systematic_resample_pytorch(weights)
        particles[:] = particles[indexes]
  
        # Step 6: Update mean
        mu = torch.sum(particles * weights.unsqueeze(1), dim=0)
        DF_t2 = time.time()

        Solver_oneloop = Solver_t2-Solver_t1
        EnSF_oneloop = EnSF_t2-EnSF_t1
        DF_oneloop = DF_t2-DF_t1

        # print(f'\t Solver one loop = {Solver_oneloop:.4f}')
        # print(f'\t EnSF one loop = {EnSF_oneloop:.4f}')
        # print(f'\t DF one loop = {DF_oneloop:.4f}')
        
        Para = mu.clone() 
 
    
    Para_lastEnSF = Para.clone()
    
    DeleLastLoop = [None]*2
    for k in range(2):
        DeleLastLoop[k] = 1 / Para_lastEnSF[k]

    MatLastLoop, Mat1dLastLoop = \
    AdvDiff_DA.Rfrt_Transport_StiffMat(subdom, nSub, nx, ny, dy, Darcy_u_perb, \
                                       dtEnSF, DeleLastLoop, len_edge_frac, delta,\
                                       1 / Para_lastEnSF[2],  poros_frt)
        
    x0filterT_last = torch.transpose(x0EnSF.clone(), 0, 1)
    
    p0lastLoop = [None]*3
    p0lastLoop[0] = x0filterT_last[RectMesh[0].NumDofus:(RectMesh[0].NumDofus+RectMesh[0].NumEms), :]
    p0lastLoop[1] = x0filterT_last[n_ori_out[0]+RectMesh[1].NumDofus:n_ori_out[0]+\
                                   RectMesh[1].NumDofus+RectMesh[1].NumEms, :]
    
    p0lastLoop[2] = x0filterT_last[n_ori_out[0]+n_ori_out[1]+2*ny:+n_ori_out[0]+n_ori_out[1]+2*ny+ny, :]

    sln_state = AdvDiff_DA.Rfrt_Transport_Solver(p0lastLoop, subdom, Mat1dLastLoop, MatLastLoop, nSub, nx, ny, \
                                                 Darcy_u_perb, dtEnSF, delta, poros_frt)

    noise_Pres = \
        np.sqrt(dtEnSF)*SDE_Sigma_Pres*torch.randn_like(x_state[:, TotalEgs+torch.arange(0, TotalEms)],\
                                                    dtype=torch.float64, device=device)
    noise_Vel = \
        np.sqrt(dtEnSF)*SDE_Sigma_Vel*torch.randn_like(x_state[:, :TotalEgs], dtype=torch.float64, device=device)

    noise_Larg = \
        0*np.sqrt(dtEnSF)*SDE_Sigma_Vel*torch.randn_like(x_state[:, TotalEgs+TotalEms+torch.arange(0, TotalLarg)], \
                                                            dtype=torch.float64, device=device)

    noise = torch.cat((noise_Vel, noise_Pres, noise_Larg), dim=1)
    x_state = torch.transpose(sln_state, 0, 1)
    
    x_state = AdvDiff_DA.RearrangeSol(x_state, RectMesh, ny)

    x_state += noise
    x_state_clone = x_state[:, spa_indices].clone()
        
    # ind_xscale1 = torch.nonzero((-1e-20 <= x_state_clone) & (x_state_clone < 1e-20), as_tuple = True)[1]
    # ind_xscale2 = torch.nonzero(((-1e-19 <= x_state_clone) & (x_state_clone < -1e-20)) | \
    #                             ((1e-20 <= x_state_clone) & (x_state_clone < 1e-19)), as_tuple= True)[1]
    
    # ind_xscale3 = torch.nonzero(((-1e-18 <= x_state_clone) & (x_state_clone < -1e-19)) | \
    #                             ((1e-19 <= x_state_clone) & (x_state_clone < 1e-18)), as_tuple= True)[1]
    # ind_xscale4 = torch.nonzero(((-1e-17 <= x_state_clone) & (x_state_clone < -1e-18)) |\
    #                             ((1e-18 <= x_state_clone) & (x_state_clone< 1e-17)), as_tuple= True)[1]
    
    # ind_xscale5 = torch.nonzero(((-1e-16 <= x_state_clone) & (x_state_clone < -1e-17)) | \
    #                             ((1e-17 <= x_state_clone) & (x_state_clone< 1e-16)), as_tuple= True)[1]
    # ind_xscale6 = torch.nonzero(((-1e-15 <= x_state_clone) & (x_state_clone < -1e-16)) | \
    #                             ((1e-16 <= x_state_clone) & (x_state_clone < 1e-15)), as_tuple= True)[1]
    # ind_xscale7 = torch.nonzero(((-1e-14 <= x_state_clone) & (x_state_clone < -1e-15)) | \
    #                             ((1e-15 <= x_state_clone) & (x_state_clone < 1e-14)), as_tuple= True)[1]
    # ind_xscale8 = torch.nonzero(((-1e-13 <= x_state_clone) & (x_state_clone < -1e-14)) |\
    #                             ((1e-14 <= x_state_clone) & (x_state_clone < 1e-13)), as_tuple= True)[1]
    # ind_xscale9 = torch.nonzero(((-1e-12 <= x_state_clone) & (x_state_clone < -1e-13)) | \
    #                             ((1e-13 <= x_state_clone) & (x_state_clone < 1e-12)), as_tuple= True)[1]
    # ind_xscale10 = torch.nonzero(((-1e-11 <= x_state_clone) & (x_state_clone < -1e-12)) | \
    #                              ((1e-12 <= x_state_clone) & (x_state_clone < 1e-11)), as_tuple= True)[1]
    # ind_xscale11 = torch.nonzero(((-1e-10 <= x_state_clone) & (x_state_clone < -1e-11)) | \
    #                              ((1e-11 <= x_state_clone) & (x_state_clone < 1e-10)), as_tuple= True)[1]
    # ind_xscale12 = torch.nonzero(((-1e-9 <= x_state_clone) & (x_state_clone < -1e-10)) | \
    #                              ((1e-10 <= x_state_clone) & (x_state_clone < 1e-9)), as_tuple= True)[1]
    # ind_xscale13 = torch.nonzero(((-1e-8 <= x_state_clone) & (x_state_clone < -1e-9)) | \
    #                              ((1e-9 <= x_state_clone) & (x_state_clone < 1e-8)), as_tuple= True)[1]
    # ind_xscale14 = torch.nonzero(((-1e-7 <= x_state_clone) & (x_state_clone < -1e-8)) | \
    #                              ((1e-8 <= x_state_clone) & (x_state_clone < 1e-7)), as_tuple= True)[1]
    # ind_xscale15 = torch.nonzero(((-1e-6 <= x_state_clone) & (x_state_clone < -1e-7)) | \
    #                              ((1e-7 <= x_state_clone) & (x_state_clone < 1e-6)), as_tuple= True)[1]
    # ind_xscale16 = torch.nonzero(((-1e-5 <= x_state_clone) & (x_state_clone < -1e-6)) | \
    #                              ((1e-6 <= x_state_clone) & (x_state_clone < 1e-5)), as_tuple= True)[1]
    # ind_xscale17 = torch.nonzero(((-1e-4 <= x_state_clone) & (x_state_clone < -1e-5)) | \
    #                              ((1e-5 <= x_state_clone) & (x_state_clone < 1e-4)), as_tuple= True)[1]
    # ind_xscale18 = torch.nonzero(((-1e-3 <= x_state_clone) & (x_state_clone < -1e-4)) | \
    #                              ((1e-4 <= x_state_clone) & (x_state_clone < 1e-3)), as_tuple= True)[1]
    # ind_xscale19 = torch.nonzero(((-1e-2 <= x_state_clone) & (x_state_clone < -1e-3)) | \
    #                              ((1e-3 <= x_state_clone) & (x_state_clone < 1e-2)), as_tuple= True)[1]


    # x_state_clone[:, ind_xscale1] *= 1e+20
    # x_state_clone[:, ind_xscale2] *= 1e+19
    # x_state_clone[:, ind_xscale3] *= 1e+18
    # x_state_clone[:, ind_xscale4] *= 1e+17
    # x_state_clone[:, ind_xscale5] *= 1e+16
    # x_state_clone[:, ind_xscale6] *= 1e+15
    # x_state_clone[:, ind_xscale7] *= 1e+14
    # x_state_clone[:, ind_xscale8] *= 1e+13
    # x_state_clone[:, ind_xscale9] *= 1e+12
    # x_state_clone[:, ind_xscale10] *= 1e+11
    # x_state_clone[:, ind_xscale11] *= 1e+10
    # x_state_clone[:, ind_xscale12] *= 1e+9
    # x_state_clone[:, ind_xscale13] *= 1e+8
    # x_state_clone[:, ind_xscale14] *= 1e+7
    # x_state_clone[:, ind_xscale15] *= 1e+6
    # x_state_clone[:, ind_xscale16] *= 1e+5
    # x_state_clone[:, ind_xscale17] *= 1e+4
    # x_state_clone[:, ind_xscale18] *= 1e+3
    # x_state_clone[:, ind_xscale19] *= 1e+2
    
    ind_xscale1 = torch.nonzero(((-1e-20 <= x_state_clone) & (x_state_clone < 0)) | \
                                ((0 <= x_state_clone) & (x_state_clone < 1e-20)), as_tuple= False)
    ind_xscale2 = torch.nonzero(((-1e-19 <= x_state_clone) & (x_state_clone < -1e-20)) | \
                                ((1e-20 <= x_state_clone) & (x_state_clone < 1e-19)), as_tuple= False)

    ind_xscale3 = torch.nonzero(((-1e-18 <= x_state_clone) & (x_state_clone < -1e-19)) | \
                                ((1e-19 <= x_state_clone) & (x_state_clone < 1e-18)), as_tuple= False)
    ind_xscale4 = torch.nonzero(((-1e-17 <= x_state_clone) & (x_state_clone < -1e-18)) |\
                                ((1e-18 <= x_state_clone) & (x_state_clone< 1e-17)), as_tuple= False)

    ind_xscale5 = torch.nonzero(((-1e-16 <= x_state_clone) & (x_state_clone < -1e-17)) | \
                                ((1e-17 <= x_state_clone) & (x_state_clone< 1e-16)), as_tuple= False)
    ind_xscale6 = torch.nonzero(((-1e-15 <= x_state_clone) & (x_state_clone < -1e-16)) | \
                                ((1e-16 <= x_state_clone) & (x_state_clone < 1e-15)), as_tuple= False)
    ind_xscale7 = torch.nonzero(((-1e-14 <= x_state_clone) & (x_state_clone < -1e-15)) | \
                                ((1e-15 <= x_state_clone) & (x_state_clone < 1e-14)), as_tuple= False)
    ind_xscale8 = torch.nonzero(((-1e-13 <= x_state_clone) & (x_state_clone < -1e-14)) |\
                                ((1e-14 <= x_state_clone) & (x_state_clone < 1e-13)), as_tuple= False)
    ind_xscale9 = torch.nonzero(((-1e-12 <= x_state_clone) & (x_state_clone < -1e-13)) | \
                                ((1e-13 <= x_state_clone) & (x_state_clone < 1e-12)), as_tuple= False)
    ind_xscale10 = torch.nonzero(((-1e-11 <= x_state_clone) & (x_state_clone < -1e-12)) | \
                                 ((1e-12 <= x_state_clone) & (x_state_clone < 1e-11)), as_tuple= False)
    ind_xscale11 = torch.nonzero(((-1e-10 <= x_state_clone) & (x_state_clone < -1e-11)) | \
                                 ((1e-11 <= x_state_clone) & (x_state_clone < 1e-10)), as_tuple= False)
    ind_xscale12 = torch.nonzero(((-1e-9 <= x_state_clone) & (x_state_clone < -1e-10)) | \
                                 ((1e-10 <= x_state_clone) & (x_state_clone < 1e-9)), as_tuple= False)
    ind_xscale13 = torch.nonzero(((-1e-8 <= x_state_clone) & (x_state_clone < -1e-9)) | \
                                 ((1e-9 <= x_state_clone) & (x_state_clone < 1e-8)), as_tuple= False)
    ind_xscale14 = torch.nonzero(((-1e-7 <= x_state_clone) & (x_state_clone < -1e-8)) | \
                                 ((1e-8 <= x_state_clone) & (x_state_clone < 1e-7)), as_tuple= False)
    ind_xscale15 = torch.nonzero(((-1e-6 <= x_state_clone) & (x_state_clone < -1e-7)) | \
                                 ((1e-7 <= x_state_clone) & (x_state_clone < 1e-6)), as_tuple= False)
    ind_xscale16 = torch.nonzero(((-1e-5 <= x_state_clone) & (x_state_clone < -1e-6)) | \
                                 ((1e-6 <= x_state_clone) & (x_state_clone < 1e-5)), as_tuple= False)
    ind_xscale17 = torch.nonzero(((-1e-4 <= x_state_clone) & (x_state_clone < -1e-5)) | \
                                 ((1e-5 <= x_state_clone) & (x_state_clone < 1e-4)), as_tuple= False)
    ind_xscale18 = torch.nonzero(((-1e-3 <= x_state_clone) & (x_state_clone < -1e-4)) | \
                                 ((1e-4 <= x_state_clone) & (x_state_clone < 1e-3)), as_tuple= False)
    ind_xscale19 = torch.nonzero(((-1e-2 <= x_state_clone) & (x_state_clone < -1e-3)) | \
                                 ((1e-3 <= x_state_clone) & (x_state_clone < 1e-2)), as_tuple= False)

    x_state_clone[ind_xscale1[:, 0], ind_xscale1[:, 1]] *= 1e+20
    x_state_clone[ind_xscale2[:, 0], ind_xscale2[:, 1]] *= 1e+19
    x_state_clone[ind_xscale3[:, 0], ind_xscale3[:, 1]] *= 1e+18
    x_state_clone[ind_xscale4[:, 0], ind_xscale4[:, 1]] *= 1e+17
    x_state_clone[ind_xscale5[:, 0], ind_xscale5[:, 1]] *= 1e+16
    x_state_clone[ind_xscale6[:, 0], ind_xscale6[:, 1]] *= 1e+15
    x_state_clone[ind_xscale7[:, 0], ind_xscale7[:, 1]] *= 1e+14
    x_state_clone[ind_xscale8[:, 0], ind_xscale8[:, 1]] *= 1e+13
    x_state_clone[ind_xscale9[:, 0], ind_xscale9[:, 1]] *= 1e+12
    x_state_clone[ind_xscale10[:, 0], ind_xscale10[:, 1]] *= 1e+11
    x_state_clone[ind_xscale11[:, 0], ind_xscale11[:, 1]] *= 1e+10
    x_state_clone[ind_xscale12[:, 0], ind_xscale12[:, 1]] *= 1e+9
    x_state_clone[ind_xscale13[:, 0], ind_xscale13[:, 1]] *= 1e+8
    x_state_clone[ind_xscale14[:, 0], ind_xscale14[:, 1]] *= 1e+7
    x_state_clone[ind_xscale15[:, 0], ind_xscale15[:, 1]] *= 1e+6
    x_state_clone[ind_xscale16[:, 0], ind_xscale16[:, 1]] *= 1e+5
    x_state_clone[ind_xscale17[:, 0], ind_xscale17[:, 1]] *= 1e+4
    x_state_clone[ind_xscale18[:, 0], ind_xscale18[:, 1]] *= 1e+3
    x_state_clone[ind_xscale19[:, 0], ind_xscale19[:, 1]] *= 1e+2
        
    sln_bar = reverse_SDE(num_indices, select_set, idA_sub, x0=x_state_clone, \
                          time_steps=euler_steps, C=1,  score_likelihood=score_likelihood)
    
    sln_bar[:, ind_scale1] /= 1e+20
    sln_bar[:, ind_scale2] /= 1e+19
    sln_bar[:, ind_scale3] /= 1e+18
    sln_bar[:, ind_scale4] /= 1e+17
    sln_bar[:, ind_scale5] /= 1e+16
    sln_bar[:, ind_scale6] /= 1e+15
    sln_bar[:, ind_scale7] /= 1e+14
    sln_bar[:, ind_scale8] /= 1e+13
    sln_bar[:, ind_scale9] /= 1e+12
    sln_bar[:, ind_scale10] /= 1e+11
    sln_bar[:, ind_scale11] /= 1e+10
    sln_bar[:, ind_scale12] /= 1e+9
    sln_bar[:, ind_scale13] /= 1e+8
    sln_bar[:, ind_scale14] /= 1e+7
    sln_bar[:, ind_scale15] /= 1e+6
    sln_bar[:, ind_scale16] /= 1e+5
    sln_bar[:, ind_scale17] /= 1e+4
    sln_bar[:, ind_scale18] /= 1e+3
    sln_bar[:, ind_scale19] /= 1e+2
        
    # sln_bar[:, ind_xscale1] /= 1e+20
    # sln_bar[:, ind_xscale2] /= 1e+19
    # sln_bar[:, ind_xscale3] /= 1e+18
    # sln_bar[:, ind_xscale4] /= 1e+17
    # sln_bar[:, ind_xscale5] /= 1e+16
    # sln_bar[:, ind_xscale6] /= 1e+15
    # sln_bar[:, ind_xscale7] /= 1e+14
    # sln_bar[:, ind_xscale8] /= 1e+13
    # sln_bar[:, ind_xscale9] /= 1e+12
    # sln_bar[:, ind_xscale10] /= 1e+11
    # sln_bar[:, ind_xscale11] /= 1e+10
    # sln_bar[:, ind_xscale12] /= 1e+9
    # sln_bar[:, ind_xscale13] /= 1e+8
    # sln_bar[:, ind_xscale14] /= 1e+7
    # sln_bar[:, ind_xscale15] /= 1e+6
    # sln_bar[:, ind_xscale16] /= 1e+5
    # sln_bar[:, ind_xscale17] /= 1e+4
    # sln_bar[:, ind_xscale18] /= 1e+3
    # sln_bar[:, ind_xscale19] /= 1e+2
    
    x_state[:, spa_indices] = sln_bar.clone()

    x_state[:, TotalEgs + torch.arange(0, NumEms[0])] =\
            torch.clamp(x_state[:, TotalEgs + torch.arange(0, NumEms[0])],\
                        min=0, max = 2.999999)

    x_state[:, TotalEgs+NumEms[0]+torch.arange(0, NumEms[1])] =\
                torch.clamp(x_state[:, TotalEgs+NumEms[0]+torch.arange(0, NumEms[1])],\
                            min=0, max = 2.999999)

    x_state[:, TotalEgs+NumEms[0]+NumEms[1]+torch.arange(0, ny)] =\
                torch.clamp(x_state[:, TotalEgs+NumEms[0]+NumEms[1]+torch.arange(0, ny)],\
                            min=0, max = 3)
    
    x_state[:, TotalEgs+TotalEms:n_dim] =\
                torch.clamp(x_state[:, TotalEgs+TotalEms:n_dim], min=0, max = 3)
    
    x_state[:, :NumDofus[0]+NumDofus[1]] =\
            torch.clamp(x_state[:, :NumDofus[0]+NumDofus[1]], min=-0.00008, max = 0.00008)

#     x_state[:, NumDofus[0]+NumDofus[1]+torch.arange(0, 2*ny)] =\
#             torch.clamp(x_state[:, NumDofus[0]+NumDofus[1]+torch.arange(0, 2*ny)], min=-0.009, max = 0.009)

    x_state[:, TotalEgs+TotalEms+NumIntEgs[0]+ny+torch.arange(0, ny)] = 3
    x_state[:, TotalEgs+TotalEms+NumIntEgs[0]+ny+ny+torch.arange(0, ny)] = 0

    x_state[:, TotalEgs+TotalEms+NumIntEgs[0]+NumBdEgs[0]-ny+NumIntEgs[1]+ny+torch.arange(0, ny)] = 3

    x_state[:, TotalEgs+TotalEms+NumIntEgs[0]+NumBdEgs[0]-ny+NumIntEgs[1]+ny+ny+torch.arange(0, ny)] = 0

    x_state[:, TotalEgs+TotalEms+NumIntEgs[0]+NumBdEgs[0]-ny+NumIntEgs[1]+NumBdEgs[1]-ny+ny-1] = 3

    x_state[:, TotalEgs+TotalEms+NumIntEgs[0]+NumBdEgs[0]-ny+NumIntEgs[1]+NumBdEgs[1]-ny+ny] = 0

    # average_values1 = (x_state[:, IntEdge2dofu[0][:, 0]]-x_state[:, IntEdge2dofu[0][:, 1]])/2
    # average_values2 = (x_state[:, NumDofus[0]+IntEdge2dofu[1][:, 0]]-x_state[:, NumDofus[0]+IntEdge2dofu[1][:, 1]])/2

    # x_statefrt = x_state[:, NumDofus[0]+NumDofus[1]+torch.arange(0, 2*ny)]
    # x_state_mid = x_statefrt[:, 1:-1]
    # average_value3 = (x_state_mid[:, 0::2]-x_state_mid[:, 1::2])/2
    # odd_indices = torch.arange(2, 2*ny-1, step=2)
    # even_indices = torch.arange(1, 2*ny-1, step=2)

    # x_state[:, NumDofus[0]+NumDofus[1]+even_indices] = average_value3
    # x_state[:, NumDofus[0]+NumDofus[1]+odd_indices] = (-1)*average_value3
    
    # x_state[:, IntEdge2dofu[0][:, 0]] = average_values1
    # x_state[:, IntEdge2dofu[0][:, 1]] = (-1)*average_values1
    
    # x_state[:, NumDofus[0]+IntEdge2dofu[1][:, 0]] = average_values2
    # x_state[:, NumDofus[0]+IntEdge2dofu[1][:, 1]] = (-1)*average_values2

    
    x_state[:, RectMesh[0].bd2dofu[subdom[0].neumann_bd_transport]] = 0
    x_state[:, NumDofus[0]+RectMesh[1].bd2dofu[subdom[1].neumann_bd_transport]] = 0

    x_state[:, RectMesh[0].bd2dofu[ny+ny+torch.arange(0, nx)]] = \
            torch.where(x_state[:, RectMesh[0].bd2dofu[ny+ny+torch.arange(0, nx)]] < 0, \
                        x_state[:, RectMesh[0].bd2dofu[ny+ny+torch.arange(0, nx)]], \
                        (-1)*x_state[:, RectMesh[0].bd2dofu[ny+ny+torch.arange(0, nx)]])

    x_state[:, NumDofus[0]+RectMesh[1].bd2dofu[ny+ny+torch.arange(0, nx)]] = \
                torch.where(x_state[:, NumDofus[0]+RectMesh[0].bd2dofu[ny+ny+torch.arange(0, nx)]] < 0, \
                            x_state[:, NumDofus[0]+RectMesh[0].bd2dofu[ny+ny+torch.arange(0, nx)]], \
                            (-1)*x_state[:, NumDofus[0]+RectMesh[0].bd2dofu[ny+ny+torch.arange(0, nx)]])
    
    # x_state[:, NumDofus[0]+NumDofus[1]] = \
    #             torch.where(x_state[:, NumDofus[0]+NumDofus[1]] < 0, \
    #                         x_state[:, NumDofus[0]+NumDofus[1]], \
    #                         (-1)*x_state[:, NumDofus[0]+NumDofus[1]])
    
    x_state = AdvDiff_DA.RearrangeSol_Reverse(x_state, RectMesh, ny)
    
    x_statebarEnSF = torch.zeros_like(est_save[[i+1], :])
    x_statebarEnSF += torch.mean(x_state.clone(),dim=0) #\bar{X^{L}}_{i+1} = \bar{X}_{i+1}
    
    
    est_save[[i+1], :] += torch.mean(x_state.clone(),dim=0)
    print(mu)
    xs.append(mu)
    diff = torch.sqrt(torch.mean((est_save[[i+1], :]  - x_statebarEnSF)**2)).item()
    rmse_temp = torch.sqrt(torch.mean((est_save[[i+1], :]  - state_EnSF[[i+1], :])**2)).item()
#     rmse_temp = torch.sqrt(torch.mean((est_save[[i+1], :]  - state_EnSF[[i+1], :])**2)).item()
    rmse = torch.sqrt(torch.mean((x_statebarEnSF  - state_EnSF[[i+1], :])**2)).item()
    
    # get time
    if x_state.device.type == 'cuda':
        torch.cuda.current_stream().synchronize()
    t2 = time.time()
    print(f'\t RMSE = {rmse_temp:.4f}')
    print(f'\t time = {t2-t1:.4f} ')

    # save information
    rmse_all.append(rmse_temp)
    # break
    if rmse_temp > 1000:
        print('diverge!')
        break

state_savepy  = state_ref.clone()
est_savepy = est_save.clone()

state_savepy = state_savepy.cpu().numpy()
est_savepy = est_savepy.cpu().numpy()
rmse_all = np.array(rmse_all)

# scipy.io.savemat('UnitedFilter_2DRft_AdvDiff_50MixedPartialOb_Noise01_ReScale_v1.mat',\
#                  {'Est_State':est_savepy, 'ExactState':state_savepy, 'rmse':rmse_all,\
#                  'nx':nx, 'ny':ny, 'nttrue': nt, 'ntEnSF': ntEnSF, 'ApprPara':xs})

# scipy.io.savemat('UnitedFilter_2DRft_AdvDiff_50MixedPartialOb_Noise0001_ReScale_v1.mat',\
#                  {'Est_State':est_savepy, 'ExactState':state_savepy, 'rmse':rmse_all,\
#                  'nx':nx, 'ny':ny, 'nttrue': nt, 'ntEnSF': ntEnSF, 'ApprPara':xs})

In [ ]:
scipy.io.savemat('UnitedFilter_2DRft_AdvDiff_50MixedPartialOb_Noise0001_ReScale_v1.mat',\
                 {'Est_State':est_savepy, 'ExactState':state_savepy, 'rmse':rmse_all,\
                 'nx':nx, 'ny':ny, 'nttrue': nt, 'ntEnSF': ntEnSF, 'ApprPara':xs})